<a href="https://colab.research.google.com/github/masifkhan3/Genetic-II/blob/main/Genetic_II.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
pip install transformers faiss-cpu datasets pandas


In [8]:
import pandas as pd

# Load your dataset
data = pd.read_csv("/content/genetic-Final.csv")

# Combine columns to form a single "text" field for retrieval purposes
data["text"] = (
    "Disease Name: " + data["Disease Name"] +
    "\nGene(s) Involved: " + data["Gene(s) Involved"] +
    "\nInheritance Pattern: " + data["Inheritance Pattern"] +
    "\nSymptoms: " + data["Symptoms"] +
    "\nSeverity Level: " + data["Severity Level"] +
    "\nRisk Assessment: " + data["Risk Assessment"] +
    "\nTreatment Options: " + data["Treatment Options"] +
    "\nSuggested Medical Tests: " + data["Suggested Medical Tests"] +
    "\nMinimum Values for Medical Tests: " + data["Minimum Values for Medical Tests"] +
    "\nEmergency Treatment: " + data["Emergency Treatment"]
)
texts = data["text"].tolist()



In [10]:
# Fill NaN values with an empty string and ensure all values are strings
data["text"] = data["text"].fillna("").astype(str)
texts = data["text"].tolist()


In [11]:
# Generate embeddings for all texts
embeddings = [embed_text(text) for text in texts]


In [21]:
import pandas as pd

# Load your dataset
data = pd.read_csv("/content/genetic-Final.csv")

# Combine columns to form a single "text" field for retrieval purposes
data["text"] = (
    "Disease Name: " + data["Disease Name"].fillna("") +
    "\nGene(s) Involved: " + data["Gene(s) Involved"].fillna("") +
    "\nInheritance Pattern: " + data["Inheritance Pattern"].fillna("") +
    "\nSymptoms: " + data["Symptoms"].fillna("") +
    "\nSeverity Level: " + data["Severity Level"].fillna("") +
    "\nRisk Assessment: " + data["Risk Assessment"].fillna("") +
    "\nTreatment Options: " + data["Treatment Options"].fillna("") +
    "\nSuggested Medical Tests: " + data["Suggested Medical Tests"].fillna("") +
    "\nMinimum Values for Medical Tests: " + data["Minimum Values for Medical Tests"].fillna("") +
    "\nEmergency Treatment: " + data["Emergency Treatment"].fillna("")
)

# Ensure all entries in `texts` are strings
texts = data["text"].fillna("").astype(str).tolist()

from transformers import AutoTokenizer, AutoModel
import torch
import faiss
import numpy as np

# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

# Function to generate embeddings
def embed_text(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state.mean(dim=1)
    return embeddings.squeeze().numpy()

# Generate embeddings for all texts
embeddings = [embed_text(text) for text in texts]

# Initialize FAISS index
embedding_dim = embeddings[0].shape[0]
index = faiss.IndexFlatL2(embedding_dim)
index.add(np.array(embeddings))


In [22]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load a generator model and tokenizer
generator_tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large")
generator_model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large")

# Function to retrieve similar entries and generate a response
def retrieve_and_generate(query, top_k=3):
    # Generate query embedding
    query_embedding = embed_text(query).reshape(1, -1)

    # Retrieve top-k similar texts
    distances, indices = index.search(query_embedding, top_k)
    retrieved_texts = [texts[i] for i in indices[0]]

    # Prepare input for generation by combining retrieved texts
    context = "\n\n".join(retrieved_texts) + "\n\nQuestion: " + query

    # Generate answer
    inputs = generator_tokenizer(context, return_tensors="pt", max_length=512, truncation=True)
    outputs = generator_model.generate(**inputs, max_length=150, num_return_sequences=1)

    # Decode the output
    return generator_tokenizer.decode(outputs[0], skip_special_tokens=True)

# Function to get user input and generate response
def ask_user():
    # Ask for input on diseases, symptoms, or medical tests
    print("Welcome to the Medical Query Assistant!")
    disease = input("Enter the disease name (or press Enter to skip): ")
    symptoms = input("Enter symptoms (or press Enter to skip): ")
    medical_test = input("Enter medical test details (or press Enter to skip): ")

    # Combine the inputs into a query
    query = ""
    if disease:
        query += f"Disease Name: {disease}\n"
    if symptoms:
        query += f"Symptoms: {symptoms}\n"
    if medical_test:
        query += f"Medical Test: {medical_test}\n"

    # Check if any query info was provided
    if not query.strip():
        print("No input provided. Please enter at least one detail.")
        return ask_user()  # Recursively call the function to get input

    # Generate a response based on the query
    response = retrieve_and_generate(query)
    print("Generated Response:", response)

# Start the interactive query session
ask_user()


Welcome to the Medical Query Assistant!
Enter the disease name (or press Enter to skip): gsd
Enter symptoms (or press Enter to skip): vomit
Enter medical test details (or press Enter to skip): 
Generated Response: Disease Name: Galactosemia                Gene(s) Involved: GALT gene fixmeInheritance Pattern: Autosomal Recessive�Symptoms: Jaundice, vomiting, lethargy, developmental delays�Severity Level: Moderate                Risk Assessment: High¯¯¯¯Treatment Options: Elimination of galactose from diet,Calcium and vitamin D supplementation                Suggested Medical Tests:   Newborn screening                Minimum Values for Medical Tests,   Glucose levels in blood and urine                Emergency Treatment: IV glucose and bicarbonate during metabolic acidosis                =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-


In [23]:
pip install torch
